In [1]:
import mdtraj as md
import numpy as np
import pickle
import os
from tqdm import tqdm_notebook
from simtk.openmm import app
from simtk.openmm import unit
import matplotlib.pyplot as plt
from pymbar import timeseries

In [2]:
def new_positions(hybrid_positions):
    n_atoms_new = htf._topology_proposal.n_atoms_new
    hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
    return hybrid_positions[hybrid_indices, :]
    
def old_positions(hybrid_positions):
    n_atoms_old = htf._topology_proposal.n_atoms_old
    hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
    return hybrid_positions[hybrid_indices, :]

def make_neq_traj(outdir, htf, i, phase, endstate, is_old=True, keep_solvent=True):
    name = 'old' if is_old else 'new'
    n_atoms = htf.old_positions(htf.hybrid_positions).shape[0] if is_old else htf.new_positions(htf.hybrid_positions).shape[0]
    
#     snapshots = np.zeros((100, n_atoms, 3))
    snapshots = []
    for j in range(100):
        if os.path.isfile(os.path.join(outdir, f"{i}_{phase}_{j}_forward_neq_{name}_waters.npy")):
            with open(os.path.join(outdir, f"{i}_{phase}_{j}_forward_neq_{name}_waters.npy"), "rb") as f:
                snapshot = np.load(f)
#                 snapshots[j] = snapshot[-1]
                snapshots.append(snapshot[-1])
    snapshots = np.array(snapshots)
    print(f"loaded {snapshots.shape[0]} snapshots")
    print(snapshots.shape)
            
#     lengths = np.array([val[i].value_in_unit_system(unit.md_unit_system) for i, val in enumerate(htf.hybrid_system.getDefaultPeriodicBoxVectors())])
#     lengths = np.array([lengths], dtype=np.float64)
#     lengths = np.repeat(lengths, snapshots.shape[0], axis=0)

#     angles = np.array([90, 90, 90])
#     angles = np.repeat(np.array([angles]), snapshots.shape[0], axis=0)
        
    if is_old:
        traj = md.Trajectory(snapshots, md.Topology.from_openmm(htf._topology_proposal.old_topology))
#         traj = md.Trajectory(snapshots, md.Topology.from_openmm(htf._topology_proposal.old_topology), unitcell_lengths=lengths, unitcell_angles=angles)
    else:
        traj = md.Trajectory(snapshots, md.Topology.from_openmm(htf._topology_proposal.new_topology))
#         traj = md.Trajectory(snapshots, md.Topology.from_openmm(htf._topology_proposal.new_topology), unitcell_lengths=lengths, unitcell_angles=angles)

    # Set unit cell vectors in traj 
    box_vectors = [val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()]
    box_vectors_formatted = np.ndarray(shape=(3,3), buffer=np.array(box_vectors)).astype(np.float32) # note that mdraj expect np.ndarray, not np.arrays
    box_vectors_repeated = np.repeat(np.array([box_vectors_formatted]), traj.xyz.shape[0], axis=0)
    traj.unitcell_vectors = box_vectors_repeated
    
    traj.image_molecules(inplace=True)
    
    if not keep_solvent:
        traj = traj.remove_solvent(exclude=['Na+', 'Cl-'])
    
    return traj
       

## 15/160

In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/15/160/"
i = 160
phase = 'complex'
endstate = 1
is_old = False
htf = pickle.load(open(os.path.join(outdir, f"{i}_{phase}.pickle"), "rb"))

INFO:rdkit:Enabling RDKit 2021.09.11 jupyter extensions


In [4]:
traj = make_neq_traj(outdir, htf, i, phase, endstate, is_old=is_old)

loaded 94 snapshots
(94, 185384, 3)


In [7]:
traj.save(os.path.join(outdir, f"{i}_{phase}_new.dcd"))
app.PDBxFile.writeFile(traj.topology.to_openmm(), traj.openmm_positions(0), file=open(os.path.join(outdir, f"{i}_{phase}_new.cif"), "w"), keepIds=True)